In [3]:
# Importing Packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [4]:
# Defining URL and Header for requesting website
URL="https://www.weather-forecast.com/countries/India-1"
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

In [5]:
#Functions for extracting Data

#For City Name
def get_city_name(soup):
    element = soup.find("li", attrs={"class": "current"})
    if element:
        return element
    else:
        return "Item not found"

#For Brief Report
def get_olrepo(soup):
    element = soup.find("div", attrs={"class": "description__text"})
    if element:
        return element
    else:
        return "Item not found"

#For Day Wise Report
def get_daysWiseData(soup):
    element = soup.find_all("p", attrs={"class": "b-forecast__table-description-content"})
    if element:
        return element
    else:
        return "Item not found"

In [7]:
# Requesting Webpage and storing in webpage named variable
webpage = requests.get(URL, headers = HEADERS)

#Converting the bytes data into html form data by using BeautifulSoup package
soup = BeautifulSoup(webpage.content, "html.parser")
# soup

In [8]:
links = soup.find_all("span", attrs={"class": "b-list-table__item-name"})
links_list=[]
# We are have storing all the in links_list named list as string
for link in links:
    val = link.find("a",href=True)['href']
    links_list.append(str(val))


In [1]:
#Creating a dictionary to store result 
dictionary = {"city":[], "olrepo":[], "oneToThreeDays": [], "fourToSevenDays": [], "sevenToTenDays": [], "nextWeek": []}

In [10]:
count = 0
#Storing only 10 result in our dictionary 
for links in links_list:
    if count == 10:
        break
    new_webPage = requests.get("https://www.weather-forecast.com"+links,headers = HEADERS)
    new_soup = BeautifulSoup(new_webPage.content, "html.parser")
    city = str(get_city_name(new_soup).text)
    dictionary["city"].append(city)
    olrepo = str(get_olrepo(new_soup).text)
    dictionary["olrepo"].append(olrepo)
    daywise = get_daysWiseData(new_soup)
    dictionary["oneToThreeDays"].append(str(daywise[0].text))
    dictionary["fourToSevenDays"].append(str(daywise[1].text))
    dictionary["sevenToTenDays"].append(str(daywise[2].text))
    dictionary["nextWeek"].append(str(daywise[3].text))
    count+=1

print(dictionary)

{'city': ['Mumbai', 'Delhi', 'Bengaluru', 'Kolkata', 'Chennai', 'Ahmedabad', 'Hyderabad', 'Pune', 'Kanpur', 'Jaipur'], 'olrepo': ['Mumbai Weather Forecast. Providing a local hourly Mumbai weather forecast of rain, sun, wind, humidity and temperature. The Long-range 12 day forecast also includes detail for Mumbai weather today. Live weather reports from Mumbai weather stations and weather warnings that include risk of thunder, high UV index and forecast gales. See the links below the 12-day Mumbai weather forecast table for other cities and towns nearby along with weather conditions for local outdoor activities.Mumbai is  5\u2009m  above sea level and located at 18.93° N 72.85° E. Mumbai has a population of 12691836. Local time in Mumbai is  IST.Read More', 'Delhi Weather Forecast. Providing a local hourly Delhi weather forecast of rain, sun, wind, humidity and temperature. The Long-range 12 day forecast also includes detail for Delhi weather today. Live weather reports from Delhi weath

In [11]:
# print(len(dictionary["city"]))
# print(len(dictionary["olrepo"]))
# print(len(dictionary["oneToThreeDays"]))
# print(len(dictionary["fourToSevenDays"]))
# print(len(dictionary["sevenToTenDays"]))
# print(len(dictionary["nextWeek"]))

In [12]:
#Using Pandas we're converting the dictionary into data frame 
weatherForeCast = pd.DataFrame.from_dict(dictionary)
#Here we're renaming the coulmn and giving the user understandable names to the column
weatherForeCast.rename(columns={'city':"City Name",
                               "olrepo": "Short Report",
                               "oneToThreeDays":"1-3 Days",
                               "fourToSevenDays":"4-7 Days",
                               "sevenToTenDays":"7-10 Days",
                               "nextWeek":"Next Week"},inplace=True)
#We're exporting the dataframe into .csv format file with index=False which denotes it won't show default indexing.
weatherForeCast.to_csv("report.csv",index=False)